## CE9010 Introduction to Data Science Project

# Data Scraper



First, we import the relevant libraries.

In [12]:
from bs4 import Comment, BeautifulSoup as bs
import urllib.request
import csv
import traceback
import numpy as np
import pandas as pd

%matplotlib inline
from IPython.display import set_matplotlib_formats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from copy import copy

# TO BE FORMATTED \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


Data Acquisition
 ------

#### Warning, long runtime of up to 40 minutes. Program will finish scraping when END OF YEAR: 2016 is printed.

Scraper written in beautifulsoup.

#### Types of data extracted
---

Our main goal of our is to predict the impact of college prospects in the first and second year of NBA based on their college stats. Hence we choose to use their advanced stats available in college (ws, ws/40min) to generalize the impact on NBA measured in their advanced stats available in the NBA (per, ws, ws/48, bpm, vorp) for their first two years in the league. We chose to extract data from 1996 as college advanced statistics were only available from the 1995 season onwards. We also chose to only include the first 30 draft picks of each draft as players beyond the 30th picks usually do not play significant minutes in their teams.

Player's NBA stats extracted from [Basketball Reference](https://www.basketball-reference.com/).

Player's College stats extracted from [Sports Reference](https://www.sports-reference.com/)

---



#### Problems encountered

---

The main problem encountered when running the scraper is the long runtime for a relatively small dataset est <200? confirm again>. The bottleneck occurs when parsing the whole page to look for the link to the sports reference website for each player in order to acquire their respective college stats. Having to parse the whole page for every player leads to a long runtime; inefficient code. A better solution would be to use another data scraping framework such as Scrapy which allows data extraction using [selectors](https://doc.scrapy.org/en/latest/topics/selectors.html). Extracting the link to sports-reference website of the page using the XPath or CSS selector would be much quicker than parsing the entire page. However as this scraper would only need to be run once it is not a major concern to optimize and rewrite another scraper.

## Utility functions

In [2]:


def extract_comments(inside_html, html_id):
    comments = inside_html.findAll(text=lambda text:isinstance(text, Comment)) #data we want is commented, hence the need 
    comments = [comment.extract() for comment in comments if 'id=\"' + html_id + '\"' in comment.extract()] #get advanced stats table
    return bs(comments[0], 'html.parser')

def read_url_into_soup(url):
    try:
        next_page = urllib.request.urlopen(url).read() # goes to player page
        return bs(next_page, 'html.parser')
    except:
        traceback.print_exc()
        return read_url_into_soup(url)

## Main

In [5]:
f = open("data/nba_all.csv", "w")
f2 = open("data/college_all.csv", "w")
writer = csv.writer(f)
writerf = csv.writer(f2)    
running = False
# year_dict = {}

for year in range(1996, 2017):
    url = "https://www.basketball-reference.com/draft/NBA_"+ str(year) + ".html"
#     req = urllib.request.Request(url)
#     html = read_url_into_soup(req)
    html = read_url_into_soup(url)
#     response = urllib.request.urlopen(req)
#     html = response.read()
#     html = bs(html, 'html.parser')
    if running:
        break
    
    for row in html.table.tbody.findAll("tr"):

        if (row.find("td")) is None:
            print("END OF YEAR: "+ str(year) +"\n")  # reach end of file
            break
            
        seasons = row.find("td", {"data-stat" : "seasons"} )
        
        if (not seasons.get_text() or  #if player did not play in NBA after being drafted
        int(seasons.get_text()) < 2 or  # if less than 2 seasons played, skip
        len(row.find("td", {"data-stat" : "college_name"}) ) < 1): #if player played in euroleague
            continue
            
        player = row.find("td",{"data-stat" : "player"})
        print(player.get_text())
        inside_html = read_url_into_soup("https://www.basketball-reference.com/" + player.a['href'])
#         next_page = urllib.request.urlopen("https://www.basketball-reference.com/" + player.a['href']).read() # goes to player page
#         inside_html = bs(next_page, 'html.parser')
        
        advanced = extract_comments(inside_html, 'advanced')
        
        # STATS
          
        out = [player.string]
        cols = ['per', 'ws_per_48', 'bpm', 'vorp'] # stats to include
        for col in cols:
            out.append(advanced.findAll('td', {'data-stat' : col})[0].string) #include both 1st and 2nd year
            out.append(advanced.findAll('td', {'data-stat' : col})[1].string) 
        out.append(year)
        writer.writerow(out)
        
        for i in inside_html.findAll("a"): #inefficient way of finding the url for college stats
            if "College Basketball" in str(i):
                coll_url = i['href']
                break
              
        coll_html = read_url_into_soup(coll_url)
#         coll_page = urllib.request.urlopen(coll_url).read()
#         coll_html = bs(coll_page, 'html.parser')

        
        coll_advanced = extract_comments(coll_html, 'players_advanced')
        coll_html = read_url_into_soup(coll_url)
        coll_players_pm = extract_comments(coll_html, 'players_per_min')
        
        
        cols = ['g','gs','fg_per_min','fga_per_min','fg_pct','fg2_per_min','fg2a_per_min','fg2_pct','fg3_per_min',
                'fg3a_per_min','fg3_pct','ft_per_min','fta_per_min','ft_pct','trb_per_min','ast_per_min', 'stl_per_min',
                'blk_per_min', 'tov_per_min','pf_per_min','pts_per_min']

        pm = [coll_players_pm.tbody.findAll('td', {'data-stat' : c})[-1].string for c in cols]
        
        #input stats
        cols = ["mp", "ts_pct","efg_pct","fg3a_per_fga_pct", "fta_per_fga_pct","ws_per_40"]
#         for col in cols:
#         ws40 = coll_advanced.tbody.findAll('td', {'data-stat' : 'ws_per_40'})[-1].string
        adv = [coll_advanced.tbody.findAll('td', {'data-stat' : c})[-1].string for c in cols]
        
        sos = coll_html.findAll('td', {'data-stat' : 'sos'})[-2].string
        result = [player.string, sos, year]
        result.extend(adv)
        result.extend(pm)
#         print(result)
        writerf.writerow(result)
#         running = True
#         break
            
f.close()
f2.close()    

Allen Iverson
Marcus Camby
Shareef Abdur-Rahim
Stephon Marbury
Ray Allen
Antoine Walker
Lorenzen Wright
Kerry Kittles
Samaki Walker
Erick Dampier
Todd Fuller
Vitaly Potapenko
Steve Nash
Tony Delk
John Wallace
Walter McCarty
Roy Rogers
Derek Fisher
Jerome Williams
Brian Evans
Priest Lauderdale
Travis Knight
END OF YEAR: 1996

Tim Duncan
Keith Van Horn
Chauncey Billups
Antonio Daniels
Tony Battie
Ron Mercer
Tim Thomas
Adonal Foyle
Danny Fortson
Tariq Abdul-Wahad
Austin Croshere
Derek Anderson
Maurice Taylor
Kelvin Cato
Brevin Knight
Johnny Taylor
Scot Pollard
Paul Grant
Anthony Parker
Ed Gray
Bobby Jackson
Rodrick Rhodes
John Thomas
Charles Smith
Jacque Vaughn
Keith Booth
END OF YEAR: 1997

Michael Olowokandi
Mike Bibby
Raef LaFrentz
Antawn Jamison
Vince Carter
Robert Traylor
Jason Williams
Larry Hughes
Paul Pierce
Bonzi Wells
Michael Doleac
Keon Clark
Michael Dickerson
Matt Harpring
Bryce Drew
Pat Garrity
Roshown McLeod
Ricky Davis
Brian Skinner
Tyronn Lue
Felipe Lopez
Sam Jacobson
Core

Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1260, in connect
    server_hostname=server_hostname)
  File "/usr/lib/python3.5/ssl.py", line 377, in wrap_socket
    _context=self)
  File "/usr/lib/python3.5/ssl.py", line 752, in __init__
    self.do_handshake()
  File "/usr/lib/python3.5/ssl.py", line 988, in do_handshake
    self._sslo

Jerome Moiso
Etan Thomas
Courtney Alexander
Mateen Cleaves
Jason Collier
Desmond Mason
Quentin Richardson
Jamaal Magloire
Speedy Claxton
Morris Peterson
Donnell Harvey
Mamadou N'Diaye
Erick Barkley
Mark Madsen
END OF YEAR: 2000

Jason Richardson
Shane Battier
Eddie Griffin
Rodney White
Joe Johnson
Kedrick Brown
Richard Jefferson
Troy Murphy
Steven Hunter
Kirk Haston
Michael Bradley
Jason Collins
Zach Randolph
Brendan Haywood
Joseph Forte
Jeryl Sasser
Brandon Armstrong
Gerald Wallace
Samuel Dalembert
Jamaal Tinsley
END OF YEAR: 2001

Mike Dunleavy
Drew Gooden
Dajuan Wagner
Chris Wilcox
Caron Butler
Jared Jeffries
Melvin Ely
Marcus Haislip
Fred Jones
Juan Dixon
Curtis Borchardt
Ryan Humphrey
Kareem Rush
Qyntel Woods
Casey Jacobsen
Tayshaun Prince
Frank Williams
John Salmons
Chris Jefferies
Dan Dickau
END OF YEAR: 2002

Carmelo Anthony
Chris Bosh
Dwyane Wade
Chris Kaman
Kirk Hinrich
T.J. Ford
Mike Sweetney
Jarvis Hayes
Nick Collison
Marcus Banks
Luke Ridnour
Reece Gaines
David West
Dahnta

Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


...

Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 1254, in do_open
    h.request(req.get_method(), req.selector, req.data, headers)
  File "/usr/lib/python3.5/http/client.py", line 1106, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  File "/usr/lib/python3.5/http/client.py", line 1252, in connect
    super().connect()
  File "/usr/lib/python3.5/http/client.py", line 849, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "/usr/lib/python3.5/socket.py", line 693, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):


END OF YEAR: 2005

LaMarcus Aldridge
Adam Morrison
Tyrus Thomas
Shelden Williams
Brandon Roy
Randy Foye
Rudy Gay
Patrick O'Bryant
J.J. Redick
Hilton Armstrong
Ronnie Brewer
Cedric Simmons
Rodney Carney
Shawne Williams
Quincy Douby
Renaldo Balkman
Rajon Rondo
Marcus Williams
Josh Boone
Kyle Lowry
Shannon Brown
Jordan Farmar
Maurice Ager
Mardy Collins
END OF YEAR: 2006

Greg Oden
Kevin Durant
Al Horford
Mike Conley
Jeff Green
Corey Brewer
Brandan Wright
Joakim Noah
Spencer Hawes
Acie Law
Thaddeus Young
Julian Wright
Al Thornton
Rodney Stuckey
Nick Young
Sean Williams
Javaris Crittenton
Jason Smith
Daequan Cook
Jared Dudley
Wilson Chandler
Morris Almond
Aaron Brooks
Arron Afflalo
Alando Tucker
END OF YEAR: 2007

Derrick Rose
Michael Beasley
O.J. Mayo
Russell Westbrook
Kevin Love
Eric Gordon
Joe Alexander
D.J. Augustin
Brook Lopez
Jerryd Bayless
Jason Thompson
Brandon Rush
Anthony Randolph
Robin Lopez
Marreese Speights
Roy Hibbert
JaVale McGee
J.J. Hickson
Ryan Anderson
Courtney Lee
Kosta 

## ML SHIT HERE
#### We used college_all and nba_all outside the data/ directory in case we accidentally ran the above function and everything is overwritten

- remove rows with NaN (somehow in 1997-1998 some college statistics were not recorded)

In [7]:
college_df = pd.read_csv('college_all.csv')
college_df.isnull().sum() #check for missing data
college_df.columns = ["Player","sos", "year","mp", "ts_pct","efg_pct","fg3a_per_fga_pct", "fta_per_fga_pct","ws_per_40",'g','gs','fg_per_min','fga_per_min','fg_pct','fg2_per_min','fg2a_per_min','fg2_pct','fg3_per_min','fg3a_per_min','fg3_pct','ft_per_min','fta_per_min','ft_pct','trb_per_min','ast_per_min', 'stl_per_min','blk_per_min', 'tov_per_min','pf_per_min','pts_per_min']

college_df = college_df.dropna(how='any').drop(columns = ['gs']) #remove column with many empty cells

nba_df = pd.read_csv('nba_all.csv', header=None)
nba_df.columns = ['Player','PER 1st', 'PER 2nd', 'WS48 1st','WS48 2nd', 'BPM 1st', 'BPM 2nd','VORP 1st', 'VORP 2nd', 'year']

nba_df = nba_df.drop(['year'], axis=1) #remove duplicate column

comb = pd.merge(nba_df, college_df, on=['Player', 'Player'])

print(college_df[college_df.isnull().any(axis=1)].shape) #amount of data to remove
print(comb.shape)
comb = comb.dropna(how = 'any') #remove rows with NaN
print(comb.shape)
print("Are there any rows with empty cells?", comb.isnull().values.any() )

# comb = comb.sort_values(by = ["College Win Shares"])


(0, 29)
(335, 37)
(335, 37)
Are there any rows with empty cells? False


# train test split

In [10]:
 #enter favourite number
seed = 100

def split_val_set(y_var, no_of_set):
    '''
    SPLITS TRAIN SET INTO VAL AND TRAINING SETS
    '''

    X_train, X_test, y_train, y_test = train_test_split(comb.iloc[:,9:], comb[y_var], test_size=0.20, random_state=seed)

    splits_x = []
    splits_y = []
    no_of_set = 4
    size_set = int(len(X_train) / no_of_set)
    for i in range(0, no_of_set - 1):
        split_set_x = X_train[size_set * i : size_set * (i+1)] #split first 3 evenly
        split_set_y = y_train[size_set * i : size_set * (i+1)]
        splits_x.append(split_set_x)
        splits_y.append(split_set_y)
    splits_x.append(X_train[size_set*3:]) 
    splits_y.append(y_train[size_set*3:])
    
    return splits_x, splits_y, X_test, y_test


# for sklearn
normalize : boolean, optional, default False

This parameter is ignored when fit_intercept is set to False. If True, the regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm. If you wish to standardize, please use sklearn.preprocessing.StandardScaler before calling fit on an estimator with normalize=False.

In [11]:
lin_reg_sklearn = LinearRegression(normalize = True) #normalize according to z

def merge_train(i, xset, yset):
    to_merge_x = [val for index, val in enumerate(xset) if index != i]
    to_merge_y = [val for index, val in enumerate(yset) if index != i]
    return pd.concat(to_merge_x), pd.concat(to_merge_y)

no_of_set = 4
for y_name in list(nba_df):
    print(y_name)
    xset , yset, x_test, y_test = split_val_set(y_name, no_of_set)
    # comb['College Win Shares per 40 min'] = preprocessing.scale(comb['College Win Shares per 40 min']
    
    
    
    print(len(xset),len(yset))

    min_mse = float('inf')
    
    for i in range(no_of_set):
        x_valid = xset[i]
        y_valid = yset[i]
        x_train, y_train = merge_train(i, xset, yset)
        
        lin_reg_sklearn.fit(x_train, y_train)
        mse = mean_squared_error(y_valid, [lin_reg_sklearn.predict(x) for x in x_valid])
        if min_mse > mse:
            min_mse = mse
            best_model = copy(lin_reg_sklearn)
        
#         mean_squared_error(y_true, y_pred, sample_weight=None, multioutput=
        w_sklearn = np.zeros([2,1])
        w_sklearn[0,0] = lin_reg_sklearn.intercept_
        w_sklearn[1,0] = lin_reg_sklearn.coef_
        print(w_sklearn)
#         plt.scatter(comb['College Win Shares per 40 min'], comb['Win Shares 2nd'],s=20, c='r', marker='o', linewidths=1)
        # plt.plot(range(-5,6), np.asarray([lin_reg_sklearn.predict(x) for x in range(-5,6)]).reshape(-1 ,1))
        # y_pred_sklearn = w_sklearn[0] + w_sklearn[1]* 
        plt.plot()
        # plt.plot(comb['College Win Shares per 40 min'], np.asarray([lin_reg_sklearn.predict(x) for x in comb['College Win Shares per 40 min']]).reshape(-1 ,1))

        plt.show()
    best_model.score()
    

Player
4 4


ValueError: could not convert string to float: 'Scot Pollard'

In [87]:
comb.head()

,Player,PER 1st,PER 2nd,Win Shares 1st,Win Shares 2nd,Win Shares per 48 min 1st,Win Shares per 48 min 2nd,BPM 1st,BPM 2nd,VORP 1st,VORP 2nd,College Win Shares,College Win Shares per 40 min,College Strength of Schedule,Year Drafted
0,Allen Iverson,18.0,20.4,4.1,9.0,0.065,0.138,1.5,3.8,2.7,4.6,8.6,0.283,9.83,1996
1,Marcus Camby,17.8,15.9,3.7,0.9,0.095,0.022,-0.3,-0.7,0.8,0.7,8.1,0.320,8.92,1996
2,Shareef Abdur-Rahim,17.4,21.1,2.9,6.9,0.049,0.113,-2.0,1.2,0.0,2.3,5.4,0.221,6.44,1996
3,Stephon Marbury,16.1,16.3,3.7,5.3,0.077,0.082,-1.0,-0.6,0.6,1.1,4.3,0.127,12.71,1996
4,Ray Allen,14.6,16.2,4.9,7.0,0.092,0.102,0.3,1.8,1.5,3.2,8.3,0.303,8.19,1996


# RECIPE TO FOLLOW

## Pre process (zero mean? unit variance?)
## Extract a subset of training data and over fit them? L train close to zero and Lval high by manually selecting hyper par
## Add regularization and evaluate the generalization performance on the validation set
## Lval and L train gap should be minimized and both be ideally small
## USe all training data and cross validation to estimate the parameters and hyper parameters 

    